### Install various pretrained models packages

##### Note: Turn on Internet on kaggle kernel

In [1]:
!pip install timm                   # - https://github.com/rwightman/pytorch-image-models
!pip install efficientnet_pytorch   # - https://github.com/lukemelas/EfficientNet-PyTorch
!pip install pretrainedmodels       # - https://github.com/Cadene/pretrained-models.pytorch

Invalid requirement: '#'
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\packaging\requirements.py", line 93, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1632, in parseString
    raise exc
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1622, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 3395, in parseImpl
    loc, exprtokens = e._parse( instring, loc, doActions )
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "

### Dependencies

In [1]:
import os
import gc
gc.enable()
import time

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from sklearn.metrics import roc_auc_score

import timm
import pretrainedmodels
import efficientnet_pytorch

In [2]:
import warnings
warnings.simplefilter("ignore")

### Load CSV Dataset

In [ ]:
train_df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sample = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

### Model

In [31]:
#This one is working but the model is too big to download in kaggle
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.arch = pretrainedmodels.__dict__["se_resnext50_32x4d"](pretrained="imagenet")
        self.arch.load_state_dict(torch.load("C:/Users/tushar.sharma/Desktop/SIIM-Kaggle/se_resnext50_32x4d-a260b3a4.pth"))
        self.arch.last_linear = nn.Linear(512, 1)
    
    def forward(self, image):
        x = self.arch(image)
        return x

In [32]:
#Issue is in this model
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.arch = pretrainedmodels.__dict__["resnet18"](pretrained="imagenet")
        self.arch.load_state_dict(torch.load("C:/Users/tushar.sharma/Desktop/SIIM-Kaggle/resnet18-5c106cde.pth"))
        self.arch.last_linear = nn.Linear(512, 1)
    
    def forward(self, image):
        x = self.arch(image)
        return x

### Image DataLoader

In [33]:
class MelanomaDataset(Dataset):
    def __init__(self, dataframe:pd.DataFrame, imfolder:str, train=bool, transforms=None):
        self.df = dataframe
        self.imfolder = imfolder
        self.train = train
        self.transforms = transforms
        
    def __len__(self):
        return self.df.__len__()
    
    def __getitem__(self, index):
        image_path = os.path.join(self.imfolder, self.df.iloc[index]['image_name'] + '.jpg')
        image = cv2.imread(image_path)
        image = transforms.ToPILImage()(image)

        if self.transforms:
            image= self.transforms(image)
        
        if self.train:
            label = self.df.iloc[index]['target']
            return image, label
        else:
            return image

### Simple Augmentation

In [34]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=256, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=32. / 255.,saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

### Average Meter Metric

In [35]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### Training Variables

In [36]:
device = "cuda"
epochs = 5
train_bs = 256

### Training

In [37]:
model = ResNet18()

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "last_linear.weight", "last_linear.bias". 

In [ ]:
train = MelanomaDataset(train_df, 
                        imfolder='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/', 
                        train=True, 
                        transforms=train_transform)
train_loader = DataLoader(dataset=train, batch_size=train_bs, shuffle=True, num_workers=2)

model = ResNet18()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3,threshold=0.001, mode="max")
criterion = nn.BCEWithLogitsLoss()

for epoch in range(epochs):
    start_time = time.time()
    model.train()    
    losses = AverageMeter()
    epoch_loss = 0.
    correct = 0
    optimizer.zero_grad()
    
    tk = tqdm(train_loader, total=len(train_loader), position=0, leave=True)
    for idx, (imgs, labels) in enumerate(tk):
        imgs_train = torch.tensor(imgs, device=device, dtype=torch.float32)
        labels_train = torch.tensor(labels, device=device, dtype=torch.float32)
        
        optimizer.zero_grad() 
        output_train = model(imgs_train)
        loss = criterion(output_train.squeeze(), labels_train)
        loss.backward()
        optimizer.step() 
        
        pred = torch.round(torch.sigmoid(output_train))  
        correct += (pred.cpu() == labels_train.cpu().unsqueeze(1)).sum().item()
        epoch_loss += loss.item()
        
        losses.update(loss.item(), imgs.size(0))
        tk.set_postfix(loss=losses.avg)
        
    train_acc = correct / len(train_idx)
    print('Epoch {:03}: | Loss: {:.3f} | Train acc: {:.3f} | Training time: {}'.format(
            epoch + 1, 
            epoch_loss, 
            train_acc,  
            str(datetime.timedelta(seconds=time.time() - start_time))[:7]))
    scheduler.step(auc)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


 12%|█▏        | 16/130 [13:59<1:24:37, 44.54s/it, loss=0.609]